<a href="https://colab.research.google.com/github/ROARMarketingConcepts/Recurrent-Neural-Network-Examples/blob/master/Deep_RNN_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### To implement a Deep RNN in TensorFlow, we create several cells and stack them into a MultiRNNCell. In the following code we stack three identical cells (but we could very well use various kinds of cells with a different number of neurons):



In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
import tensorflow as tf

In [0]:
reset_graph()

n_inputs = 2
n_steps = 5

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [4]:
n_neurons = 100
n_layers = 3

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
          for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [0]:
init = tf.global_variables_initializer()

In [0]:
X_batch = np.random.rand(2, n_steps, n_inputs)

In [0]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch})

In [10]:
outputs_val.shape

(2, 5, 100)

### Distributing a Deep RNN Across Multiple GPUs

In [0]:
class DeviceCellWrapper(tf.contrib.rnn.RNNCell):
  def __init__(self, device, cell):
    self._cell = cell
    self._device = device

  @property
  def state_size(self):
    return self._cell.state_size

  @property
  def output_size(self):
    return self._cell.output_size

  def __call__(self, inputs, state, scope=None):
    with tf.device(self._device):
        return self._cell(inputs, state, scope)

In [0]:
reset_graph()

n_inputs = 5
n_steps = 20
n_neurons = 100

X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])

In [0]:
devices = ["/cpu:0", "/cpu:0", "/cpu:0"] # replace with ["/gpu:0", "/gpu:1", "/gpu:2"] if you have 3 GPUs
cells = [DeviceCellWrapper(dev,tf.contrib.rnn.BasicRNNCell(num_units=n_neurons))
         for dev in devices]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [0]:
init = tf.global_variables_initializer()

In [19]:
with tf.Session() as sess:
    init.run()
    print(sess.run(outputs, feed_dict={X: np.random.rand(2, n_steps, n_inputs)}))

[[[-3.2902010e-02  1.1701923e-01 -4.5899246e-02 ...  3.8447704e-02
   -5.2490000e-02  5.8043696e-02]
  [ 3.5428837e-02 -7.4261218e-02 -6.4237513e-02 ... -2.4575704e-01
   -1.7762682e-01 -3.2653331e-03]
  [ 2.5972652e-01  1.4460522e-01  3.2931221e-01 ...  3.3882204e-01
    5.9885412e-02 -8.3144858e-02]
  ...
  [ 1.1804418e-01 -2.4166366e-01 -8.2779825e-01 ... -1.8578218e-01
    1.8968213e-01 -1.3526212e-01]
  [ 2.8783119e-01  3.9295399e-01 -3.1523648e-01 ... -7.8560874e-02
    4.3272454e-01 -6.6182777e-02]
  [ 7.4978012e-01  6.1105335e-01 -3.0168572e-01 ...  1.9565013e-01
   -1.6245714e-01 -5.2396566e-02]]

 [[-6.6802777e-02  7.3557667e-02 -2.9027537e-02 ... -7.7996828e-02
   -3.6577322e-02 -4.6675194e-02]
  [ 1.7154643e-01  4.2841923e-03  2.1881519e-03 ...  3.6853828e-02
   -8.1324145e-02  9.0100162e-02]
  [ 3.4563683e-02 -9.5752515e-02  1.5945031e-01 ...  1.6302212e-01
   -6.8304457e-02  1.0799057e-01]
  ...
  [ 1.9641407e-01 -2.1523033e-01 -5.8253795e-01 ...  3.0253306e-01
    5.4786